In [1]:
import pandas as pd
import pulp

In [2]:
# Step 1: Load and Prepare Data
# ----------------------------
df = pd.read_csv('players.csv')
df

,Player ID,Name,Role,Overseas,Batting Average,Strike Rate,Bowling Economy,Wickets,Catches,Composite Score
0,1,Shreyas Iyer,Batsman,No,32.613545,134.027646,NaN,NaN,111.0,78.715066
1,2,Nehal Wadhera,Batsman,No,24.960000,134.800000,NaN,NaN,10.0,54.920000
2,3,Harnoor Singh,Batsman,No,10.000000,89.280000,NaN,NaN,1.0,31.984000
3,4,Pyla Avinash,Batsman,No,20.250000,117.390000,NaN,NaN,2.0,45.742000
4,5,Shashank Singh,Batsman,No,24.080000,143.030000,NaN,NaN,22.0,59.349000
5,6,Vishnu Vinod,Wicket-Keeper,No,31.150000,142.230000,NaN,NaN,26.0,51.306000
6,7,Prabhsimran Singh,Wicket-Keeper,No,31.220000,145.760000,NaN,NaN,32.0,54.440000
7,8,Josh Inglis,Wicket-Keeper,Yes,30.416650,149.523111,NaN,NaN,112.0,86.871282
8,9,Arshdeep Singh,Bowler,No,NaN,NaN,8.414650,306.0,52.0,192.317070
9,10,Yuzvendra Chahal,Bowler,No,NaN,NaN,7.756945,460.0,84.0,291.248611


In [3]:
# Drop rows with missing composite scores (e.g., Musheer Khan)
df = df.dropna(subset=['Composite Score'])


In [4]:
# Convert data types
df['Player ID'] = df['Player ID'].astype(int)
df['Composite Score'] = df['Composite Score'].astype(float)

C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\2510840357.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Player ID'] = df['Player ID'].astype(int)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\2510840357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Composite Score'] = df['Composite Score'].astype(float)


In [5]:
# Create a binary flag for Overseas (1 if Yes, 0 if No)
df['OverseasFlag'] = df['Overseas'].apply(lambda x: 1 if x.strip().lower() == 'yes' else 0)

C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\1680432684.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['OverseasFlag'] = df['Overseas'].apply(lambda x: 1 if x.strip().lower() == 'yes' else 0)


In [6]:
# Create binary flags for roles based on the Role column
df['IsWicketkeeper'] = (df['Role'].str.strip().str.lower() == 'wicket-keeper').astype(int)
df['IsBatsman']     = (df['Role'].str.strip().str.lower() == 'batsman').astype(int)
df['IsBowler']      = (df['Role'].str.strip().str.lower() == 'bowler').astype(int)
df['IsAllrounder']  = (df['Role'].str.strip().str.lower() == 'all-rounder').astype(int)


C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\1703518403.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IsWicketkeeper'] = (df['Role'].str.strip().str.lower() == 'wicket-keeper').astype(int)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\1703518403.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['IsBatsman']     = (df['Role'].str.strip().str.lower() == 'batsman').astype(int)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_13532\1703518403.py:4: SettingWithCopyWarning: 
A value is trying t

In [7]:
# ----------------------------
# Step 2: Create Dictionaries for Model Data
# ----------------------------
scores = dict(zip(df['Player ID'], df['Composite Score']))
overseas = dict(zip(df['Player ID'], df['OverseasFlag']))
wicketkeeper = dict(zip(df['Player ID'], df['IsWicketkeeper']))
batsman = dict(zip(df['Player ID'], df['IsBatsman']))
bowler = dict(zip(df['Player ID'], df['IsBowler']))
allrounder = dict(zip(df['Player ID'], df['IsAllrounder']))

players = list(df['Player ID'])

In [8]:
# ----------------------------
# Step 3: Define the Optimization Model
# ----------------------------
# Create a maximization problem
model = pulp.LpProblem("Select_Playing_XI", pulp.LpMaximize)

# Decision variables: x[i] = 1 if player i is selected, else 0
x = pulp.LpVariable.dicts('x', players, lowBound=0, upBound=1, cat=pulp.LpBinary)

# Objective Function: Maximize total composite score
model += pulp.lpSum([scores[i] * x[i] for i in players]), "Total_Composite_Score"

# Constraint 1: Exactly 11 players must be selected
model += pulp.lpSum([x[i] for i in players]) == 11, "Total_Players"

# Constraint 2: Exactly 4 overseas players
model += pulp.lpSum([overseas[i] * x[i] for i in players]) == 4, "Overseas_Players"

# Constraint 3: At least 1 wicketkeeper
model += pulp.lpSum([wicketkeeper[i] * x[i] for i in players]) >= 1, "AtLeastOne_Wicketkeeper"

# Constraint 4: At least 3 batsmen
model += pulp.lpSum([batsman[i] * x[i] for i in players]) >= 3, "AtLeastThree_Batsmen"

# Constraint 5: At least 3 bowlers
model += pulp.lpSum([bowler[i] * x[i] for i in players]) >= 3, "AtLeastThree_Bowlers"

# Constraint 6: At least 1 all-rounder
model += pulp.lpSum([allrounder[i] * x[i] for i in players]) >= 1, "AtLeastOne_Allrounder"


In [9]:
# ----------------------------
# Step 4: Solve the Model
# ----------------------------
model.solve()

1

In [10]:
# ----------------------------
# Step 5: Retrieve and Display the Solution
# ----------------------------
selected_players = [i for i in players if pulp.value(x[i]) == 1]
print("Selected Player IDs:", selected_players)
# Display detailed info for selected players by merging with the original dataframe
selected_df = df[df['Player ID'].isin(selected_players)]
print(selected_df)

Selected Player IDs: [1, 2, 5, 7, 8, 9, 10, 12, 15, 21, 22]
    Player ID               Name           Role Overseas  Batting Average  \
0           1       Shreyas Iyer        Batsman       No        32.613545   
1           2      Nehal Wadhera        Batsman       No        24.960000   
4           5     Shashank Singh        Batsman       No        24.080000   
6           7  Prabhsimran Singh  Wicket-Keeper       No        31.220000   
7           8        Josh Inglis  Wicket-Keeper      Yes        30.416650   
8           9     Arshdeep Singh         Bowler       No              NaN   
9          10   Yuzvendra Chahal         Bowler       No              NaN   
11         12       Yash Thakur          Bowler       No              NaN   
14         15    Lockie Ferguson         Bowler      Yes              NaN   
20         21     Marcus Stoinis    All-Rounder      Yes        30.321307   
21         22      Glenn Maxwell    All-Rounder      Yes        28.471730   

    Strike Rate